# Mobile_ALOHA:从VAE到ACT


## 从GAN, VAE到扩散模型diffusion model：

    VAE常作为一种生成模型, 此外还有生成对抗网络GAN，以及现在非常火热的扩散模型diffusion model等。GAN中，生成器和判别器通过一个对抗过程进行训练。生成器试图生成逼真的数据样本，而判别器则试图区分生成的数据和真实数据。生成器的目标是欺骗判别器，使其认为生成的数据是真实的；判别器的目标是准确地区分真实数据和生成数据。这个过程可以看作是一个两者之间的博弈。事实上，因为GAN的目标函数就是用来以假乱真的，所以GAN生成的图片的保真度非常高，但其需要同时对抗性训练两个网络，不够稳定，而且创造性不够，且整个训练过程所有输出来自于网络，是隐式的，并不是一个概率模型，可解释性不高，在数学上不如后续的AE，DAE，VAE优美。
    
    接着让我们来看AutoEncoder（AE），以及后来的VAE。AE相对简单，也是很早之前的技术了，大概就是：给定一个输入X，经过一个Encoder，得到一个向量（bottleneck），然后这个bottleneck再输入给一个Decoder，试图去重建输入的X,因为是X自己重建自己，有一种自回归的意味，所以叫自编码器（AutoEncoder）。紧接着出来一个denosing auto-encoder（DAE），就是把输入的原图X进行了一定程度的打乱，再把扰乱过后的Xc（corrupted X)输入到encoder，后续与AE一样，在最后我们希望输出的X依然能够重建原始输入的X，而不是扰乱过后的Xc，这个改进被证明非常有用。
![Image](./1.jpeg)

    但是不论是AE还是DAE还是MAE，他们的主要目的都是去学中间的这个bottleneck特征向量Z，然后拿这个特征去做一些分类，检测，分割的任务，而不是用来做生成的，因为其实它学到的不是一个概率分布，我们没法对他进行采样，也就是这里的Z，它并不是像GAN的一样是一个随机噪声，而是一个专门用来重建的一个特征，但是这种Encoder_Decoder是一种很好的结构，那问题就是我们如何使用这种结构去做图像生成呢？那么我们就有了VAE变自分编码器，Variational Auto_Encoder.
![Image](./2.jpg)

    VAE和AE其实是非常不一样的，虽然它的整体框架看起来差不多，然后它的目标函数还是让最后的输出去尽量重建输入的X，但是重要的区别在于，它的中间不再是学习一个固定的bottleneck特征向量，而是一个分布，在这里作者假设它服从一个高斯分布（原因后面会说到），在这里我们encoder就是一些FC层，然后去预测这个高斯分布的均值和方差，那么我们的Z的分布就可以根据上面的公式从得出，之后我们就可以从这个分布中进行采样并输入Decoder，也就是说，当我们的模型训练好之后，你完全可以前面的这个Encoder直接扔掉，将采样到的Z放入Decoder，得到输出，这就可以来做图像生成了。因为VAE预测的是一个分布，从贝叶斯概率的角度来看，前面这个给定X得到Z的过程，就是一个后验概率，然后学出来的distribution就是一个先验分布，那对于decoder部分，对于给定的Z，去预测一张图片X，其实就是似然，那么目标函数我们就是要做一个最大似然估计，从数学上看，就是非常干净优美。
    

## VAE的直觉理解

    

## 证据下界ELBO

## 目标函数 Objective

## 重参数化 Reparameterization

![Image](./reparameter.jpeg)

## 损失函数 Loss function

![image](./loss.jpeg)

## VAE 效果可视化

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

    def loss_function(self, recon_x, x, mu, logvar):
        BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return BCE + KLD

In [5]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = model.loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')

    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')

In [6]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += model.loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), 'reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print(f'====> Test set loss: {test_loss:.4f}')

In [7]:
from torchvision.utils import save_image

def sample(model, epoch):
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'sample_' + str(epoch) + '.png')

batch_size = 128
epochs = 10
no_cuda = False
seed = 1
log_interval = 10

cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda" if cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

model = VAE(input_dim=784, hidden_dim=400, latent_dim=20).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    #sample(model, epoch)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████| 9912422/9912422 [00:10<00:00, 919782.88it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 150867.44it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████| 1648877/1648877 [00:01<00:00, 900885.82it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 1100614.06it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.596191
Train Epoch: 1 [12800/60000 (21%)]	Loss: 177.746704
Train Epoch: 1 [25600/60000 (43%)]	Loss: 154.853241
Train Epoch: 1 [38400/60000 (64%)]	Loss: 139.441666
Train Epoch: 1 [51200/60000 (85%)]	Loss: 132.236008
====> Epoch: 1 Average loss: 164.8239
====> Test set loss: 128.4906
Train Epoch: 2 [0/60000 (0%)]	Loss: 129.827499
Train Epoch: 2 [12800/60000 (21%)]	Loss: 128.262863
Train Epoch: 2 [25600/60000 (43%)]	Loss: 123.591225
Train Epoch: 2 [38400/60000 (64%)]	Loss: 122.722702
Train Epoch: 2 [51200/60000 (85%)]	Loss: 119.248871
====> Epoch: 2 Average loss: 122.2261
====> Test set loss: 116.3133
Train Epoch: 3 [0/60000 (0%)]	Loss: 112.561386
Train Epoch: 3 [12800/60000 (21%)]	Loss: 108.714340
Train Epoch: 3 [25600/60000 (43%)]	Loss: 113.825851
Train Epoch: 3 [38400/60000 (64%)]	Loss: 110.753777
Train Epoch: 3 [51200/60000 (85%)]	Loss: 116.306938
====> Epoch: 3 Average

第1轮训练后重建的图像效果
![Image](./reconstruction_1.png)

第10轮训练后重建的图像效果
![Image](./reconstruction_10.png)

可以看出模型经过训练后重建的图像更加接近输入的原图像，且loss有收敛的趋势

In [1]:
sample(model,epochs) #对训练完的模型decoder构造随机的输入进行采样生成来查看效果

NameError: name 'sample' is not defined

![Image](./sample_10.png)
